In [7]:
import os
import pandas as pd

def starts_with_20(filename):
    """Check if the filename starts with '20'."""
    return filename.startswith("20")

def get_csv_files(folder_path):
    """Return a list of CSV files in the folder that start with '20'."""
    all_files = os.listdir(folder_path)
    csv_files = [file for file in all_files if file.endswith(".csv") and starts_with_20(file)]
    return csv_files

def read_and_combine_csv_files(folder_path, csv_files):
    """Read and concatenate the CSV files into a single DataFrame."""
    combined_dataframe = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in csv_files])
    return combined_dataframe

In [8]:
folder_path = "data/"

# Get a list of CSV files in the folder that start with "20"
csv_files = get_csv_files(folder_path)

# Read and concatenate the CSV files into a single DataFrame
combined_dataframe = read_and_combine_csv_files(folder_path, csv_files)

# Print the combined DataFrame
combined_dataframe

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,weather,rain,wind_gust
0,1419984000,1419927716,1419976557,24.52,24.62,1015,61,16.51,0,10000.0,3.6,320,"[{'id': 501, 'main': 'Rain', 'description': 'm...",{'1h': 1.44},NaN
1,1420070400,1420014152,1420062976,25.27,25.39,1016,59,16.69,0,10000.0,3.1,330,"[{'id': 800, 'main': 'Clear', 'description': '...",NaN,NaN
2,1420156800,1420100590,1420149394,25.10,25.36,1016,65,18.06,0,10000.0,2.6,300,"[{'id': 800, 'main': 'Clear', 'description': '...",NaN,NaN
3,1420243200,1420187029,1420235810,24.06,24.48,1018,75,19.35,0,10000.0,5.7,170,"[{'id': 211, 'main': 'Thunderstorm', 'descript...",NaN,NaN
4,1420329600,1420273468,1420322226,22.51,22.96,1019,82,19.28,75,9000.0,2.6,120,"[{'id': 803, 'main': 'Clouds', 'description': ...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,1609027200,1608970791,1609019680,19.83,20.01,1016,82,16.67,75,6000.0,3.6,140,"[{'id': 701, 'main': 'Mist', 'description': 'm...",NaN,NaN
172,1609113600,1609057225,1609106103,20.83,21.19,1017,85,18.22,20,8000.0,1.5,70,"[{'id': 701, 'main': 'Mist', 'description': 'm...",NaN,NaN
173,1609200000,1609143660,1609192525,20.50,20.93,1018,89,18.63,75,3000.0,2.1,20,"[{'id': 502, 'main': 'Rain', 'description': 'h...",{'1h': 11.94},NaN
174,1609286400,1609230096,1609278946,20.46,20.91,1016,90,18.76,75,10000.0,1.0,100,"[{'id': 701, 'main': 'Mist', 'description': 'm...",NaN,NaN
